# the all new scrappy

In [ ]:
import requests
import json
import re
import pandas as pd
# import numpy as np
import datetime
import plotly.express as px


class Scrappy():
    
    def __init__(self, funds_overview_path=None, timeseries_path=None):
        """
        Parameters
        ----------
        funds_overview_path: str, optional
            (relative) path to funds summary file
            
        timeseries_path: str, optional
            (relative) path to timeseries data file
            
        Returns
        -------
        None
        """
        def str_to_list(s):
            return list(eval(s))
        
        if not funds_overview_path:
            self.funds_overview = pd.DataFrame()
        else:
            self.funds_overview = pd.read_csv(funds_overview_path, index_col='isin',
                                              converters={'performance': str_to_list, 'risk': str_to_list})
            
        if not timeseries_path:
            self.ts = None
        else:
            self.ts = pd.read_csv(timeseries_path, index_col='x')
            self.ts.index = pd.to_datetime(self.ts.index)
            
        self.api_url = 'https://www.fondsdiscount.de/themes/barcelona/content/module/chart/getChartData.php'
        self.headers = {
            'Host': 'www.fondsdiscount.de',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
            'Accept': '*/*',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate, br',
            'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'X-Requested-With': 'XMLHttpRequest',
            'Content-Length': '68',
            'Origin': 'https://www.fondsdiscount.de',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Pragma': 'no-cache',
            'Cache-Control': 'no-cache',
            'TE': 'Trailers',}
        self.data_template = {
            'function': 'getMultiChartIsin',
            'isin': 'DE000A0KEYM4',
            'range': '4',
            'ind': '',
            'charttyp': 'p',}
        
        self.session = None
        
    def select_funds(self, filter_lst=None, column='index'):
        """returns full or filtered funds overview from previous search or import of funds overview.
        
        Parameters
        ----------
        filter_lst: {str, list(str)}, optional
            filter fonds summary by string or list of strings. if not specified, gives full fonds summary
        
        column: str, default='index'
            which column in self.fonds_overview to filter by. if 'index', filter by index
        
        Returns
        -------
        pandas.DataFrame
            full or filtered fonds summary
        
        """
        if filter_lst is None:
            return self.funds_overview
        else:
            if isinstance(filter_lst, str):
                filter_lst = [filter_lst]
                
            if column == 'index':
                try:
                    return self.funds_overview.loc[filter_lst]
                except KeyError:
                    print("KeyError, returning only found ISINs")
                    return self.funds_overview.loc[self.funds_overview.index.intersection(filter_lst)]
            else:
                return self.funds_overview[self.funds_overview[column].isin(filter_lst)]
            
    def search_funds(self, regex_str, column='index'):
        if column == 'index':
            return self.funds_overview.filter(regex=regex_str, axis=0)
        else:
            return self.funds_overview.loc[self.funds_overview[column].str.contains(regex_str, regex=True), :]
        
    def restart_session(self):
        if self.session is not None:
            self.session.close()
            
        self.session = requests.Session()
        return self.session
        
    def get_response(self, url, data=None, params=None, headers=None, method='POST'):
        """returns HTTP response of successful request. Automatic retry on ConnectionError
        
        Parameters
        ----------
        url, data, params, headers, method='POST'
            see requests.request
            
        Returns
        -------
        requests.Response, None
            if successful (status_code = 200)
        """
        if self.session is None:
            self.restart_session()
        try:
            response = self.session.request(method=method, url=url, data=data, params=params, headers=headers)
        except requests.ConnectionError:
            print('ConnectionError, restarting session.')
            self.restart_session()
            response = self.session.request(method=method, url=url, data=data, params=params, headers=headers)
        
        if response.status_code == 200:
            return response
        
    def parse_timeseries_data(self, response_str):
        """Clean up and parse http response of timeseries into DataFrame.
        
        Parameters
        ----------
        response_str: str
            response of http request as string
            
        Returns
        -------
        pandas.DataFrame, None
            parsed response string
        
        """
        reg = re.search(r"id: \'([A-Z0-9]{12})\'", response_str)
        if reg is not None:
            isin = reg.group(1)
            s = re.search(r'data: (\[.+\])\}', response_str).group(1) # find data
            s = s.replace('Date.UTC', 'datetime.date')
            s = re.sub(r',0(\d)', r',\1', s) # delete leading zeros in dates
            df = pd.DataFrame.from_records(eval(s), columns=['x', isin])
            df.set_index('x', inplace=True)
            df.index = pd.to_datetime(df.index)
            return df
    
    def get_timeseries(self, isins):
        """ Gets timeseries data from fondsdiscounter website. Sets self.ts
        
        Parameters
        ----------
        isins: str, list(str)
            List of ISINs
            
        Returns
        -------
        pandas.DataFrame
            full timeseries for ISINs in wide format
            
        TODO
        ----
        remove old ts if new data queried
        """
        
        if isinstance(isins, str):
            isins = [isins]
        else:
            isins = list(isins)
        
        df = None
        data = self.data_template.copy()
        num_isins = len(isins)
        for i, isin in enumerate(isins):
            print('{} / {}'.format(i, num_isins), end='\r')
            data['isin'] = isin
            response = self.get_response(url=self.api_url, data=data, headers=self.headers, method='POST')
            if response:
                parsed = self.parse_timeseries_data(response.text)
                if parsed is not None:
                    if df is None:
                        df = parsed
                    else:
                        df = df.join(parsed, how='outer')
                else:
                    print('No data for {}'.format(isin))
            else:
                print('No response for {}'.format(isin))
#         self.ts = self.ts.join(df, how='outer')
        self.ts = self.ts.combine_first(df)
        return df
    
    def save_timeseries(self, filename='timeseries.csv'):
        if self.ts is not None:
            self.ts.to_csv(filename)

    def get_search_results(self, filter_ids=[5634,5703,5582], etf=False, benchmark=False, limit=100):
        """Gets funds and ETF summary data from onvista website. Appends results to self.funds_overview.
        
        Parameters
        ----------
        filter_ids: int, list(int), None, default=[5634,5703,5582]
            onvista IDs to filter for, e.g.
            5634: Aktien Nachhaltigkeit
            5703: Aktien Branche Alternative Energien
            5582: Aktien Branche Ökologie, Umwelttechnologien
            
        etf: bool, default=False
            if True, returns ETFs. if False, returns funds
            
        benchmark: bool, default=False
            if True, returns benchmark indices for S&P500, NASDAQ, FTSE100, Dow Jones, DAX & MSCI World as ETF
            
        limit: int, default=100
            maximum search results
            
        Returns
        -------
        pandas.DataFrame
            results of current search query
        """
        
        self.search_api_url = 'https://api.onvista.de/api/v1/funds/finder/configuration_query'
        params = {
            'application': 'WEBSITE',
            'device': 'DESKTOP',
            'order': 'DESC',
            'page': '0',
            'perPage': str(limit),
            'queryParameters': '',
            'sort': 'performancePct1Y',}
        headers = {
            'Host': 'api.onvista.de',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
            'Accept': 'application/json',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate, br',
            'Content-Type': 'application/json',
            'Origin': 'https://www.onvista.de',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Pragma': 'no-cache',
            'Cache-Control': 'no-cache',
            'TE': 'Trailers',}
        
        cols = ['instrument.entitySubType', 'fundsDetails.nameInvestmentFocus',
                'fundsDetails.fundsInvestmentRegion.name', 'issuer.nameGroupIssuer', 'benchmark.instrument.name',
                'instrument.name', 'instrument.isin', 'fundsBaseData.nameCountry', 'fundsBaseData.isoCurrencyFund',
                'fundsBaseData.volumeFund', 'fundsDetails.fundsTypeCapitalisation.name',
                'fundsBaseData.maxPctInitialFee', 'fundsBaseData.ongoingCharges',
                'fundsEvaluation.morningstarRating', 'fundsEvaluation.morningstarRating3y', 'fundsEvaluation.morningstarRating5y',
                'fundsEvaluation.feriRating', 'fundsEvaluation.riskClass',
                'fundsPerformanceList.list', 'fundsRiskList.list']
        colnames = ['type', 'investment_focus', 'region', 'issuer', 'benchmark', 'name', 'isin', 'country', 'currency', 'volume',
                    'fund_type', 'initial_fee', 'TER', 'morningstar', 'morningstar_3y', 'morningstar_5y', 'feri', 'risk_class',
                    'performance', 'risk']
        
        options = list()
        if etf:
            options.append('isExchangeTraded=true')
        
        if filter_ids is not None:
            if isinstance(filter_ids, int):
                filter_ids = str(filter_ids)
            elif isinstance(filter_ids, list):
                filter_ids = ','.join(map(str, filter_ids))
            options.append('idInvestmentFocus=' + filter_ids)
            
        if benchmark:
            # ignore other options if benchmark=True
            params['queryParameters'] = 'isExchangeTraded=true&idInstrumentBenchmark=16204403,4646272,83327,376508,376391,376376&idTypeReplication=2'
        else:
            params['queryParameters'] = '&'.join(options)
                
        response = self.get_response(url=self.search_api_url, params=params, headers=headers, method='GET')
        
        df = pd.json_normalize(response.json()['list'])
        df = df[cols]
        df.columns = colnames
        df.set_index('isin', inplace=True)
                
        self.funds_overview = pd.concat([self.funds_overview, df], axis=0)
        # remove duplicates by index:
        self.funds_overview = self.funds_overview[~self.funds_overview.index.duplicated(keep='first')]
        return df
    
    def save_search_results(self, filename='funds_overview.csv'):
        if self.funds_overview is not None:
            self.funds_overview.to_csv(filename)
            
    def select_ts(self, columns):
        """return timeseries for ISINs if they are present in self.ts.
        
        Parameters
        ----------
        columns: list(str)
            list of ISINs to select from timeseries
        
        Returns
        -------
        pandas.DataFrame
            timeseries which could be found
        """
        print('Not in ts:', end=' ')
        cols = filter(None, [i if i in set(self.ts.columns) else print(i, end= ',') for i in columns])
        return self.ts[cols]
    
    def get_ts_long(self, ts=None, val_name='y', var_name='name'):
        """transform timeseries into long format for plotting
        
        Parameters
        ----------
        ts: pandas.DataFrame, optional
            timeseries dataframe. if None, self.ts is used
            
        val_name: str, default='y'
            name of the resulting function values column
            
        var_name: str, default='name'
            name for the variable column in long format e.g. ISINs
            
        Returns
        -------
        pandas.DataFrame
            timeseries in long format
        """
        if ts is None:
            ts = self.ts
        id_var = ts.index.name
        ts_long = pd.melt(ts.reset_index(), id_vars=id_var, var_name=var_name, value_name=val_name)
        return ts_long
    
    def transform(self, func, ts=None, args=(), **kwds):
        """apply function to timeseries
        
        Parameters
        ----------
        func: function
            function, which gets applied to each timeseries
            
        ts: pandas.DataFrame, optional
            timeseries where fuction should be applied. if None, self.ts is used
            
        args, kwds: optional
            (keyword) arguments to be applied to func
        """
        if ts is None:
            ts = self.ts
        return ts.apply(func, result_type=None, args=args, **kwds)

    def plot_ts(self, columns=None, transform_func=None, legend=False, name=None):
        """plots timeseries for 
        
        Parameters
        ----------
        columns: list(str)
            list of ISINs to plot
            
        transform_func: function, optional
            function to apply to timeseries before plotting
            
        legend: bool, default=False
            if True, plots legend
            
        name: str, optional
            column of self.funds_overview to use for timeseries labels
        """
        if columns is not None:
            ts = self.select_ts(columns)
        else:
            ts = self.ts
            
        if name is not None:
            try:
                ts.columns = self.funds_overview.loc[ts.columns, name]
            except KeyError:
                # timeseries queried from different source, thus ISIN can be missing from self.funds_overview
                ts.columns = [self.funds_overview[name].get(i, i) for i in ts.columns]
        
        x = ts.index.name
        y = 'y'
        color = 'name'
        
        if transform_func is not None:
            # need to interpolate NaN so that timeseries don't get lost upon transformation
            ts = self.transform(transform_func, ts=ts.interpolate('time'))
        
        ts_long = self.get_ts_long(ts, val_name=y, var_name=color).dropna()

        fig = px.line(ts_long, x=x, y=y, color=color)
#         fig.update_layout(legend={'yanchor': 'top', 'xanchor': 'left', 'y': 1, 'x': 0})
        fig.update_layout(showlegend=legend)
        fig.show()
    

s = Scrappy('funds_overview.csv', 'timeseries.csv')

pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

benchmarks = ['IE00B5BMR087', 'DE000A0F5UF5', 'IE0005042456', 'DE0006289390', 'DE0005933931', 'LU0340285161']
favs = ['AT0000A03N37',
        'AT0000A28E70', # same but cheaper
        'LU0557290854',
        'LU2016064201', # neu und teuer
        'LU2016063229', # bisschen schlechter aber billiger
        'LU0302446306',
        'BE0948502365',
        'LU0301152442', # teuer
        'LU0348927095',
        'LU0232467083',
        'LU0428702855', # teurer
        'LU0969484418',
#         'DE000A0LGNP3',
        'IE00BDCY2C68',
       ]

In [ ]:
df = s.get_search_results(benchmark=True)
df = s.get_search_results(limit=2000, etf=False)
df = s.get_search_results(limit=2000, etf=True)

s.save_search_results()

In [ ]:
# df = s.funds_overview
# pd.DataFrame(df.loc[0, 'fundsFigureBenchmarkList.list']).join(
#     pd.DataFrame(df.loc[0, 'fundsPerformanceList.list']), rsuffix='_', how='outer').join(
#         pd.DataFrame(df.loc[0, 'fundsRiskList.list']), rsuffix='__', how='outer')

pd.DataFrame(s.funds_overview.at[s.funds_overview.index[0], 'risk'])

In [ ]:
s.plot_ts(benchmarks, lambda s: s/s.loc['2020-01-17'], name='name')

In [ ]:
s.plot_ts(favs, lambda s: s/s.loc['2020-01-17'], name='name')
s.select_funds(favs)

In [ ]:
s.select_ts(favs).corr()

In [ ]:
timeidx = 3
perf = s.funds_overview.join(s.funds_overview[['risk']].apply(lambda s: pd.Series(s[0][timeidx]), axis=1)).sort_values(by='maxDrawdown', ascending=False).head(50)
best = list(perf.index)
perf

In [ ]:
s.plot_ts(benchmarks + best, lambda s: s/s.loc['2020-01-17'], name='name', legend=True)

In [ ]:
s.search_funds('Gifford', 'name').head(50)

In [ ]:
s.plot_ts(s.search_funds('Gifford', 'name').index, lambda s: s/s.loc['2020-01-17'], name='name', legend=True)

In [ ]:
oeko = list(s.select_funds('Branche: Ökologie / Umwelttechnologie Aktien', 'investment_focus').index)
alt = list(s.select_funds('Branche: Alternative Energien Aktien', 'investment_focus').index)
s.plot_ts(alt + benchmarks, lambda s: s/s.loc['2020-01-17'], name='name')

In [ ]:
p = ['IE00BFNM3P36', 'IE00B52VJ196']
s.plot_ts(p + benchmarks, lambda s: s/s.loc['2020-01-17'], name='name')

In [ ]:
etf = list(s.select_funds('ETF', 'type').index)
s.plot_ts(etf + benchmarks, lambda s: s/s.loc['2020-01-17'], name='name')

In [ ]:
perf = s.transform(lambda s: s/s.loc['2020-01-17']).sort_values(by='2010-01-01', axis=1).iloc[:,:50]
best = list(perf.columns)
s.select_funds(best)
# perf['2010']

In [ ]:
old_favs = ['LU0823414809',
        'LU1136261358',
        'IE00BDCY2C68',
        'LU0405860593',
        'AT0000A28E70',
        'LU2016064201',
       ]

s.plot_ts(old_favs + benchmarks, lambda s: s/s.loc['2020-01-17'], name='name')

# old scrappy
died because api shut down...

In [ ]:
import requests
import json
import dirtyjson
import re
import pandas as pd
import numpy as np
import datetime

from selenium import webdriver
from selenium.webdriver.firefox.options import Options

class Scrappy():
    
    def __init__(self, fonds_overview_path=None):
        """
        Parameters
        ----------
        fonds_overview_path: str
            (relative) path to fonds summary file
            
        Returns
        -------
        None
        """
        if not fonds_overview_path:
            self.fonds_overview = pd.DataFrame(columns=['name', 'data_id'])
        else:
            self.fonds_overview = pd.read_csv(fonds_overview_path, dtype={'data_id': int})
            self.fonds_overview.index = self.fonds_overview['name']
        
        self.api_url = 'https://www.onvista.de/fonds/ajax/getChartData'
        self.headers_template = {'Host': 'www.onvista.de',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
            'Accept': '*/*',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate, br',
            'Referer': 'https://www.onvista.de/fonds/{fonds_name}',
            'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'X-Requested-With': 'XMLHttpRequest',
            'Content-Length': '460',
            'Origin': 'https://www.onvista.de',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Pragma': 'no-cache',
            'Cache-Control': 'no-cache'}
        self.data_template = 'chartType=snapshotchart&values={options}'
        self.options = {"main":{"class":"ui card active selected",
#                                 "offsetStartRange":"#*#",
                                "codeResolution":"1D",
                                "codeTypeOffset":"TD",
                                "span":"Y",
                                "tickinterval":"1",
                                "idNotation":"38038227",
                                "tspan":"",
                                "datetimeTzStartRange":"2006-04-27",
                                "datetimeTzEndRange":"2020-06-11",
                                "fundBenchmark":"0",
                                "withEarnings":"1"
                        }}
        
        self.search_api_url = 'https://www.onvista.de/fonds/ajax/searchresult'
        self.search_headers_template = {'Host': 'www.onvista.de',
            'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
            'Accept': "application/json, text/javascript, */*; q=0.01",
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate, br',
            'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'X-Requested-With': 'XMLHttpRequest',
            'Content-Length': '7411',
            'Origin': 'https://www.onvista.de',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Referer': 'https://www.onvista.de/fonds/finder?idInvestmentFocus={filter_focus}'}
        self.search_data_template = "draw=6&columns[0][data]=nsin&columns[0][name]=&columns[0][searchable]=true&columns[0][orderable]=true&columns[0][search][value]=&columns[0][search][regex]=false&columns[1][data]=nameInstrument&columns[1][name]=&columns[1][searchable]=true&columns[1][orderable]=true&columns[1][search][value]=&columns[1][search][regex]=false&columns[2][data]=nameGroupIssuer&columns[2][name]=&columns[2][searchable]=true&columns[2][orderable]=true&columns[2][search][value]=&columns[2][search][regex]=false&columns[3][data]=nameTypeFund&columns[3][name]=&columns[3][searchable]=true&columns[3][orderable]=true&columns[3][search][value]=&columns[3][search][regex]=false&columns[4][data]=nameInvestmentFocus&columns[4][name]=&columns[4][searchable]=true&columns[4][orderable]=true&columns[4][search][value]=&columns[4][search][regex]=false&columns[5][data]=volumeFundEuro&columns[5][name]=&columns[5][searchable]=true&columns[5][orderable]=true&columns[5][search][value]=&columns[5][search][regex]=false&columns[6][data]=performancePctCY&columns[6][name]=&columns[6][searchable]=true&columns[6][orderable]=true&columns[6][search][value]=&columns[6][search][regex]=false&columns[7][data]=performancePct1Y&columns[7][name]=&columns[7][searchable]=true&columns[7][orderable]=true&columns[7][search][value]=&columns[7][search][regex]=false&columns[8][data]=performancePct3Y&columns[8][name]=&columns[8][searchable]=true&columns[8][orderable]=true&columns[8][search][value]=&columns[8][search][regex]=false&columns[9][data]=performancePct5Y&columns[9][name]=&columns[9][searchable]=true&columns[9][orderable]=true&columns[9][search][value]=&columns[9][search][regex]=false&columns[10][data]=performanceYearCY&columns[10][name]=&columns[10][searchable]=true&columns[10][orderable]=true&columns[10][search][value]=&columns[10][search][regex]=false&columns[11][data]=performanceYear1Y&columns[11][name]=&columns[11][searchable]=true&columns[11][orderable]=true&columns[11][search][value]=&columns[11][search][regex]=false&columns[12][data]=performanceYear3Y&columns[12][name]=&columns[12][searchable]=true&columns[12][orderable]=true&columns[12][search][value]=&columns[12][search][regex]=false&columns[13][data]=performanceYear5Y&columns[13][name]=&columns[13][searchable]=true&columns[13][orderable]=true&columns[13][search][value]=&columns[13][search][regex]=false&columns[14][data]=volatilityCY&columns[14][name]=&columns[14][searchable]=true&columns[14][orderable]=true&columns[14][search][value]=&columns[14][search][regex]=false&columns[15][data]=volatility1Y&columns[15][name]=&columns[15][searchable]=true&columns[15][orderable]=true&columns[15][search][value]=&columns[15][search][regex]=false&columns[16][data]=volatility3Y&columns[16][name]=&columns[16][searchable]=true&columns[16][orderable]=true&columns[16][search][value]=&columns[16][search][regex]=false&columns[17][data]=volatility5Y&columns[17][name]=&columns[17][searchable]=true&columns[17][orderable]=true&columns[17][search][value]=&columns[17][search][regex]=false&columns[18][data]=maxDrawdownCY&columns[18][name]=&columns[18][searchable]=true&columns[18][orderable]=true&columns[18][search][value]=&columns[18][search][regex]=false&columns[19][data]=maxDrawdown1Y&columns[19][name]=&columns[19][searchable]=true&columns[19][orderable]=true&columns[19][search][value]=&columns[19][search][regex]=false&columns[20][data]=maxDrawdown3Y&columns[20][name]=&columns[20][searchable]=true&columns[20][orderable]=true&columns[20][search][value]=&columns[20][search][regex]=false&columns[21][data]=maxDrawdown5Y&columns[21][name]=&columns[21][searchable]=true&columns[21][orderable]=true&columns[21][search][value]=&columns[21][search][regex]=false&columns[22][data]=countMonthsPositivePerformanceCY&columns[22][name]=&columns[22][searchable]=true&columns[22][orderable]=true&columns[22][search][value]=&columns[22][search][regex]=false&columns[23][data]=countMonthsPositivePerformance1Y&columns[23][name]=&columns[23][searchable]=true&columns[23][orderable]=true&columns[23][search][value]=&columns[23][search][regex]=false&columns[24][data]=countMonthsPositivePerformance3Y&columns[24][name]=&columns[24][searchable]=true&columns[24][orderable]=true&columns[24][search][value]=&columns[24][search][regex]=false&columns[25][data]=countMonthsPositivePerformance5Y&columns[25][name]=&columns[25][searchable]=true&columns[25][orderable]=true&columns[25][search][value]=&columns[25][search][regex]=false&columns[26][data]=maxPctInitialFee&columns[26][name]=&columns[26][searchable]=true&columns[26][orderable]=true&columns[26][search][value]=&columns[26][search][regex]=false&columns[27][data]=maxPctManagementFee&columns[27][name]=&columns[27][searchable]=true&columns[27][orderable]=true&columns[27][search][value]=&columns[27][search][regex]=false&columns[28][data]=ongoingCharges&columns[28][name]=&columns[28][searchable]=true&columns[28][orderable]=true&columns[28][search][value]=&columns[28][search][regex]=false&columns[29][data]=isZeroPercentFund&columns[29][name]=&columns[29][searchable]=true&columns[29][orderable]=false&columns[29][search][value]=&columns[29][search][regex]=false&columns[30][data]=nameTypeCapitalisation&columns[30][name]=&columns[30][searchable]=true&columns[30][orderable]=true&columns[30][search][value]=&columns[30][search][regex]=false&columns[31][data]=compare&columns[31][name]=&columns[31][searchable]=true&columns[31][orderable]=false&columns[31][search][value]=&columns[31][search][regex]=false&order[0][column]=0&order[0][dir]=asc&start=0&length={limit}&search[value]=&search[regex]=false&params=idInvestmentFocus={filter_focus}"
        
        self.etf_search_api_url = "https://www.onvista.de/etf/ajax/finder"
        self.etf_search_headers_template = {"Host": "www.onvista.de",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
            "Accept": "application/json, text/javascript, */*; q=0.01",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip, deflate, br",
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "X-Requested-With": "XMLHttpRequest",
            "Content-Length": "7514",
            "Origin": "https://www.onvista.de",
            "DNT": "1",
            "Connection": "keep-alive",
            # "Referer": "https://www.onvista.de/etf/finder?etfIndustry%5B%5D=Nachhaltigkeit&etfType=&currentQuote=on&maxPctManagementFee%5Be%5D=on&ongoingCharges%5Be%5D=on&volumeFundEuro%5Be%5D=on&earning%5Be%5D=on&performancePct1Y%5Be%5D=on&correlation1Y%5Be%5D=on&beta1Y%5Be%5D=on&volatility1M%5Be%5D=on&performancePct6M%5Be%5D=on&alpha3M%5Be%5D=on&sharpeRatio1Y%5Be%5D=on&volatility1Y%5Be%5D=on&performancePct1M%5Be%5D=on&alpha1Y%5Be%5D=on&sharpeRatio3M%5Be%5D=on&relativeReturnPct3M%5Be%5D=on&standardDeviation3M%5Be%5D=on&beta3M%5Be%5D=on&trackingError3M%5Be%5D=on&relativeReturnPct1Y%5Be%5D=on&standardDeviation1Y%5Be%5D=on&maxDrawdown1Y%5Be%5D=on&trackingError1Y%5Be%5D=on&correlation3M%5Be%5D=on&filterKey=etfIndustry%5B%5D&filterValue=Nachhaltigkeit",
            "Pragma": "no-cache",
            "Cache-Control": "no-cache",
            "TE": "Trailers"}
        self.etf_search_data_template = "draw=39&columns%5B0%5D%5Bdata%5D=nsin&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=nameInstrument&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=nameGroupIssuer&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=etfRegion&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=currentQuote&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=false&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=maxPctManagementFee&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=true&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=ongoingCharges&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=true&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B7%5D%5Bdata%5D=volumeFundEuro&columns%5B7%5D%5Bname%5D=&columns%5B7%5D%5Bsearchable%5D=true&columns%5B7%5D%5Borderable%5D=true&columns%5B7%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B7%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B8%5D%5Bdata%5D=earning&columns%5B8%5D%5Bname%5D=&columns%5B8%5D%5Bsearchable%5D=true&columns%5B8%5D%5Borderable%5D=true&columns%5B8%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B8%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B9%5D%5Bdata%5D=alpha3M&columns%5B9%5D%5Bname%5D=&columns%5B9%5D%5Bsearchable%5D=true&columns%5B9%5D%5Borderable%5D=true&columns%5B9%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B9%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B10%5D%5Bdata%5D=alpha1Y&columns%5B10%5D%5Bname%5D=&columns%5B10%5D%5Bsearchable%5D=true&columns%5B10%5D%5Borderable%5D=true&columns%5B10%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B10%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B11%5D%5Bdata%5D=beta3M&columns%5B11%5D%5Bname%5D=&columns%5B11%5D%5Bsearchable%5D=true&columns%5B11%5D%5Borderable%5D=true&columns%5B11%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B11%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B12%5D%5Bdata%5D=beta1Y&columns%5B12%5D%5Bname%5D=&columns%5B12%5D%5Bsearchable%5D=true&columns%5B12%5D%5Borderable%5D=true&columns%5B12%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B12%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B13%5D%5Bdata%5D=correlation3M&columns%5B13%5D%5Bname%5D=&columns%5B13%5D%5Bsearchable%5D=true&columns%5B13%5D%5Borderable%5D=true&columns%5B13%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B13%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B14%5D%5Bdata%5D=correlation1Y&columns%5B14%5D%5Bname%5D=&columns%5B14%5D%5Bsearchable%5D=true&columns%5B14%5D%5Borderable%5D=true&columns%5B14%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B14%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B15%5D%5Bdata%5D=maxDrawdown1Y&columns%5B15%5D%5Bname%5D=&columns%5B15%5D%5Bsearchable%5D=true&columns%5B15%5D%5Borderable%5D=true&columns%5B15%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B15%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B16%5D%5Bdata%5D=performancePct1M&columns%5B16%5D%5Bname%5D=&columns%5B16%5D%5Bsearchable%5D=true&columns%5B16%5D%5Borderable%5D=true&columns%5B16%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B16%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B17%5D%5Bdata%5D=performancePct6M&columns%5B17%5D%5Bname%5D=&columns%5B17%5D%5Bsearchable%5D=true&columns%5B17%5D%5Borderable%5D=true&columns%5B17%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B17%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B18%5D%5Bdata%5D=performancePct1Y&columns%5B18%5D%5Bname%5D=&columns%5B18%5D%5Bsearchable%5D=true&columns%5B18%5D%5Borderable%5D=true&columns%5B18%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B18%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B19%5D%5Bdata%5D=relativeReturnPct3M&columns%5B19%5D%5Bname%5D=&columns%5B19%5D%5Bsearchable%5D=true&columns%5B19%5D%5Borderable%5D=true&columns%5B19%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B19%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B20%5D%5Bdata%5D=relativeReturnPct1Y&columns%5B20%5D%5Bname%5D=&columns%5B20%5D%5Bsearchable%5D=true&columns%5B20%5D%5Borderable%5D=true&columns%5B20%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B20%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B21%5D%5Bdata%5D=sharpeRatio3M&columns%5B21%5D%5Bname%5D=&columns%5B21%5D%5Bsearchable%5D=true&columns%5B21%5D%5Borderable%5D=true&columns%5B21%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B21%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B22%5D%5Bdata%5D=sharpeRatio1Y&columns%5B22%5D%5Bname%5D=&columns%5B22%5D%5Bsearchable%5D=true&columns%5B22%5D%5Borderable%5D=true&columns%5B22%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B22%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B23%5D%5Bdata%5D=standardDeviation3M&columns%5B23%5D%5Bname%5D=&columns%5B23%5D%5Bsearchable%5D=true&columns%5B23%5D%5Borderable%5D=true&columns%5B23%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B23%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B24%5D%5Bdata%5D=standardDeviation1Y&columns%5B24%5D%5Bname%5D=&columns%5B24%5D%5Bsearchable%5D=true&columns%5B24%5D%5Borderable%5D=true&columns%5B24%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B24%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B25%5D%5Bdata%5D=trackingError3M&columns%5B25%5D%5Bname%5D=&columns%5B25%5D%5Bsearchable%5D=true&columns%5B25%5D%5Borderable%5D=true&columns%5B25%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B25%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B26%5D%5Bdata%5D=trackingError1Y&columns%5B26%5D%5Bname%5D=&columns%5B26%5D%5Bsearchable%5D=true&columns%5B26%5D%5Borderable%5D=true&columns%5B26%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B26%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B27%5D%5Bdata%5D=volatility1M&columns%5B27%5D%5Bname%5D=&columns%5B27%5D%5Bsearchable%5D=true&columns%5B27%5D%5Borderable%5D=true&columns%5B27%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B27%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B28%5D%5Bdata%5D=volatility1Y&columns%5B28%5D%5Bname%5D=&columns%5B28%5D%5Bsearchable%5D=true&columns%5B28%5D%5Borderable%5D=true&columns%5B28%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B28%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=18&order%5B0%5D%5Bdir%5D=desc&start=0&length={limit}&search%5Bvalue%5D=&search%5Bregex%5D=false&params=etfIndustry%255B%255D%3D{filter_value}%26etfType%3D%26currentQuote%3Don%26maxPctManagementFee%255Be%255D%3Don%26ongoingCharges%255Be%255D%3Don%26volumeFundEuro%255Be%255D%3Don%26earning%255Be%255D%3Don%26performancePct1Y%255Be%255D%3Don%26correlation1Y%255Be%255D%3Don%26beta1Y%255Be%255D%3Don%26volatility1M%255Be%255D%3Don%26performancePct6M%255Be%255D%3Don%26alpha3M%255Be%255D%3Don%26sharpeRatio1Y%255Be%255D%3Don%26volatility1Y%255Be%255D%3Don%26performancePct1M%255Be%255D%3Don%26alpha1Y%255Be%255D%3Don%26sharpeRatio3M%255Be%255D%3Don%26relativeReturnPct3M%255Be%255D%3Don%26standardDeviation3M%255Be%255D%3Don%26beta3M%255Be%255D%3Don%26trackingError3M%255Be%255D%3Don%26relativeReturnPct1Y%255Be%255D%3Don%26standardDeviation1Y%255Be%255D%3Don%26maxDrawdown1Y%255Be%255D%3Don%26trackingError1Y%255Be%255D%3Don%26correlation3M%255Be%255D%3Don%26filterKey%3DetfIndustry%255B%255D%26filterValue%3D{filter_value}"

    def get_funds(self, filter_lst=None, column='isin'):
        """returns full or filtered fonds summary
        
        Parameters
        ----------
        filter_lst: {str, list(str)}, optional
            filter fonds summary by string or list of strings. if not specified, gives full fonds summary
        
        column: str, default='isin'
            which column in self.fonds_overview to filter by
        
        Returns
        -------
        pandas.DataFrame
            full or filtered fonds summary
        
        """
        if filter_lst is None:
            return self.fonds_overview
        else:
            if isinstance(filter_lst, str):
                filter_lst = [filter_lst]
            return self.fonds_overview[self.fonds_overview[column].isin(filter_lst)]
        
    def get_response(self, url, data, headers):
        response = requests.post(url=url, data=data, headers=headers)
        if response.status_code == 200:
            return response
        
    def parse_data(self, response_str):
        """response from http request is not in readable json format. 
        
        Parameters
        ----------
        response_str: str
            response of http request as string
            
        Returns
        -------
        pandas.DataFrame
            parsed response string
        
        """
        def row_to_date(row):
            return datetime.datetime(int(row[0]), int(row[1])+1, int(row[2]))
        
        regex_wrap = r"(Date.UTC\([0-9, ]+\))"
        s = re.sub(regex_wrap, r"'\1'", response_str)
        df = pd.DataFrame.from_dict(dirtyjson.loads(s)[0]['data'], orient='index')
        temp = df['x'].str.extract(r'([\d]{4}), ([\d]{1,2}), ([\d]{1,2})')
        df['x'] = temp.apply(row_to_date, axis=1)
        return df

    def get_data(self, fonds_name, data_id=None, time_range=[datetime.timedelta(days=366), 'today']):
        """
        params:
            id_notation: get from website
            fonds_name: valid fonds name, from get_search_results()['name']
            time_range: [start_time, end_time]
                start_time: 'created': from beginning of fonds
                            datetime.timedelta(): end_date - timedelta
                            iso-format: '2000-01-01'
                end_time: 'today'
                          iso-format: '2000-01-01'
        """
        options = self.options.copy()
            
        if data_id:
            pass
        elif all(self.fonds_overview.loc[self.fonds_overview['name']==fonds_name, 'data_id'].notnull()):
            data_id = self.fonds_overview.loc[self.fonds_overview['name']==fonds_name,'data_id'].values[0]
            if data_id == 0:
                print('No data_id found')
                return
        else:
            data_ids = self.get_ids([fonds_name])
            if len(data_ids) == 1:
                data_id = data_ids[0]
            else:
                print('No data_id found')
                return
        
        options['main']['idNotation'] = str(int(data_id))
        
        if time_range[1] == 'today':
            end_time = datetime.date.today().isoformat()
        else:
            end_time = time_range[1]
        options['main']['datetimeTzEndRange'] = end_time
            
        if isinstance(time_range[0], datetime.timedelta):
            start_time = (datetime.date.fromisoformat(end_time) - time_range[0]).isoformat()
        elif time_range[0] == 'created':
            options['main']['offsetStartRange'] = '#*#'
            start_time = '2000-01-01' # can be arbitrary, but has to be set
        else:
            start_time = time_range[0]
        options['main']['datetimeTzStartRange'] = start_time
        
        data = self.data_template.format(options=json.dumps(options))
        
        headers = self.headers_template.copy()
        headers['Referer'] = headers['Referer'].format(fonds_name=fonds_name)
        headers['Content-Length'] = str(len(data))
        
        response = self.get_response(self.api_url, data, headers)
        if response is not None:
            result = self.parse_data(response.text)
            return result
        
    def get_search_results(self, filter_focus=5634, limit=100):
        """
        filter_focus:
            5634: Aktien Nachhaltigkeit
            5703: Aktien Branche Alternative Energien
            5582: Aktien Branche Ökologie, Umwelttechnologien
        """
        
        def parse_number(series):
            return series.str.extract(r'([-,\d]+)').replace(r'(,)','.', regex=True).astype(float)
        
        data = self.search_data_template.format(filter_focus=filter_focus,
                                                limit=limit)
        headers = self.search_headers_template.copy()
        headers['Referer'] = headers['Referer'].format(filter_focus=filter_focus)
        headers['Content-Length'] = str(len(data))
        
        response = self.get_response(self.search_api_url, data, headers)
        if response is not None:
            result_raw = pd.DataFrame(response.json()['data']).replace('n.a.', np.nan)
            result_raw = result_raw[result_raw['nameInstrument'].notnull()]
            result = pd.DataFrame(index=result_raw.index)
            result['name'] = result_raw['nameInstrument'].str.extract(r'<a href="\/fonds\/([\S]+)"')
            result['nsin'] = result_raw['nsin']
            result['isin'] = result['name'].apply(lambda s: s.split('-')[-1])
            result['initial_fee'] = parse_number(result_raw['maxPctInitialFee'])
            # result['management_fee'] = parse_number(result_raw['maxPctManagementFee'])
            result['ongoing_charges'] = parse_number(result_raw['ongoingCharges'])
            result['capitalization'] = result_raw['nameTypeCapitalisation']
            result['type'] = result_raw['nameTypeFund']
            result['investment_focus'] = result_raw['nameInvestmentFocus']
            result['volume'] = parse_number(result_raw['volumeFundEuro'])            
            result['performance_cy'] = parse_number(result_raw['performancePctCY'])
            result['performance_1y'] = parse_number(result_raw['performancePct1Y'])
            result['performance_3y'] = parse_number(result_raw['performancePct3Y'])
            result['performance_5y'] = parse_number(result_raw['performancePct5Y'])
            result['volatility_cy'] = parse_number(result_raw['volatilityCY'])
            result['volatility_1y'] = parse_number(result_raw['volatility1Y'])
            result['volatility_3y'] = parse_number(result_raw['volatility3Y'])
            result['volatility_5y'] = parse_number(result_raw['volatility5Y'])
            result['max_drawdown_cy'] = parse_number(result_raw['maxDrawdownCY'])
            result['max_drawdown_1y'] = parse_number(result_raw['maxDrawdown1Y'])
            result['max_drawdown_3y'] = parse_number(result_raw['maxDrawdown3Y'])
            result['max_drawdown_5y'] = parse_number(result_raw['maxDrawdown5Y'])
            result['months_positive_cy'] = parse_number(result_raw['countMonthsPositivePerformanceCY'])
            result['data_id'] = np.nan
            result.index = result['name']
            self.fonds_overview = pd.concat([self.fonds_overview, result], axis=0).drop_duplicates()
            return result
        
    def get_etf_search_results(self, filter_value="Nachhaltigkeit", limit=100):
        
        def parse_number(series):
            return series.replace('n.a.', np.nan)

        data = self.etf_search_data_template.format(filter_value=filter_value, limit=limit)
        headers = self.etf_search_headers_template.copy()
        headers['Content-Length'] = str(len(data))

        response = self.get_response(self.etf_search_api_url, data, headers)
        if response is not None:
            result_raw = pd.DataFrame(response.json()['data'][1:]) # first entry is advertisement
            result = pd.DataFrame(index=result_raw.index)
            result['name'] = result_raw['url']
            result['nsin'] = result_raw['nsin']
            result['isin'] = result_raw['url'].apply(lambda s: s.split('-')[-1])
            # result['management_fee'] = parse_number(result_raw['maxPctManagementFee'])
            result['ongoing_charges'] = parse_number(result_raw['ongoingCharges'])
            result['capitalization'] = "Ausschüttend"
            result['type'] = "ETF"
            result['investment_focus'] = result_raw['etfRegion']
            result['capitalization_amount'] = result_raw['earning']
            result['capitalization_currency'] = result_raw['isoCurrencyFund']
            result['performance_1m'] = parse_number(result_raw['performancePct1M'])
            result['performance_6m'] = parse_number(result_raw['performancePct6M'])
            result['performance_1y'] = parse_number(result_raw['performancePct1Y'])
            result['volatility_1m'] = parse_number(result_raw['volatility1M'])
            result['volatility_1y'] = parse_number(result_raw['volatility1Y'])
            result['max_drawdown_1y'] = parse_number(result_raw['maxDrawdown1Y'])
            result['outperformance_3m'] = parse_number(result_raw['relativeReturnPct3M'])
            result['outperformance_1y'] = parse_number(result_raw['relativeReturnPct1Y'])
            result['data_id'] = np.nan
            result.index = result['name']
            self.fonds_overview = pd.concat([self.fonds_overview, result], axis=0).drop_duplicates()
            return result
    
    def dump_search_results(self, filename='fonds_overview.csv'):
        self.fonds_overview.to_csv(filename, index=False)
    
    def get_ids(self, fonds_names=None):
        if not fonds_names:
            fonds_names = self.fonds_overview['name']
        options = Options() 
        options.set_headless(True)
        options.set_preference('javascript.enabled', False)
        with webdriver.Firefox(executable_path='/home/boris/git/scrappy/src/geckodriver',
                               options=options) as driver:
            data_ids = list()
            for fonds_name in fonds_names:
                if all(self.fonds_overview.loc[self.fonds_overview['name']==fonds_name,'data_id'].isnull()):
                    driver.get('https://www.onvista.de/fonds/{fonds_name}'.format(fonds_name=fonds_name))
                    source = driver.find_element_by_id('select-exchange').get_attribute('innerHTML')
                    p = re.compile(r'"(\d+)"')
                    m = p.search(source)
                    if m:
                        data_id = m.group(1)
                        data_ids.append(data_id)
                    else:
                        data_id = 0
                        print('No data_id for ', fonds_name)
                    self.fonds_overview.loc[self.fonds_overview['name']==fonds_name,'data_id'] = data_id
        return data_ids
    
    def get_timeseries(self, fonds_names, time_range, long=False):
        if long == True:
            self.ts_long = pd.DataFrame(columns=['x', 'y', 'name'])
        else:
            self.ts = pd.DataFrame(columns=['x'])
        
        for fonds_name in fonds_names:
            print(fonds_name)
            data = self.get_data(fonds_name, time_range=time_range)
            if data is None:
                print('No data for {}'.format(fonds_name))
                continue
            x = data[['x','y']]
#             x['isin'] = self.fonds_overview.loc[self.fonds_overview['name']==fonds_name, 'isin'].values[0]
            x['y'] = x['y'] / x.loc['1','y']
            if long == True:
                x['name'] = fonds_name
                self.ts_long = self.ts_long.append(x, ignore_index=True)
            else:
                x.columns = ['x', fonds_name]
                self.ts = pd.merge(self.ts, x, on='x', how='outer')
                
        if long == True:
            return self.ts_long
        else:
            return self.ts.sort_values(by='x')
    
    def get_ts_long(self, id_var='x'):
        if self.ts_long:
            return self.ts_long
        else:
            fonds_names = list(self.ts.columns)
            fonds_names.remove(id_var)
            self.ts_long = pd.melt(self.ts, id_vars=[id_var], value_vars=fonds_names, var_name='name', value_name='y')
            return self.ts_long

s = Scrappy('fonds_overview2.csv')

# export fonds list

s = Scrappy()
s.get_search_results(limit=2000)
s.get_search_results(filter_focus=5703)
s.get_search_results(filter_focus=5582)
s.get_etf_search_results()
s.get_ids() # takes a long time!
s.dump_search_results()

In [ ]:
# take best performers in different categories

indicators = ['performance_cy', 'performance_1y', 'performance_3y', 'performance_5y', 'performance_1m', 'performance_6m']
max_best = 15

best_fonds = pd.DataFrame(columns=s.fonds_overview.columns)
for ind in indicators:
    b = s.fonds_overview.sort_values(by=ind, ascending=False).head(max_best)
    best_fonds = best_fonds.append(s.fonds_overview.sort_values(by=ind, ascending=False).head(max_best))
best_fonds = best_fonds.drop_duplicates().sort_index()
best_fonds

## plotting

In [ ]:
import plotly.express as px
# import plotly.graph_objects as go

def get_ts_long(ts_wide, id_var='x'):
    fonds_names = list(ts_wide.columns)
    fonds_names.remove(id_var)
    return pd.melt(ts_wide, id_vars=[id_var], value_vars=fonds_names, var_name='name', value_name='y')

def plot_ts(ts):
    fig = px.line(get_ts_long(ts), x='x', y='y', color='name')
    # fig.update_layout(legend={'yanchor': 'top', 'xanchor': 'left', 'y': 1, 'x': 0})
    fig.update_layout(showlegend=False)
    fig.show()

In [ ]:
time_range = ['2020-02-19', 'today']

X = s.get_timeseries(best_fonds['name'], time_range)
X

In [ ]:
# rev adjusted by initial fees

init_fees = 1.-best_fonds['initial_fee'].fillna(0)/100.

Y = X.copy()
for f in best_fonds['name']:
    Y[f] = X[f] * init_fees[f]

plot_ts(Y)

In [ ]:
Y = X.copy()
for f in best_fonds['name']:
    last = (Y[f].dropna()).iloc[-1]
    Y[f] = last/Y[f]
    
plot_ts(Y)

In [ ]:
best_fonds

In [ ]:
favs = ['BNPP-ENERGY-TRANSITION-I-EUR-ACC-Fonds-LU0823414809',
        'GREEN-BENEFIT-GLOBAL-IMPACT-FUND-I-EUR-DIS-Fonds-LU1136261358',
        'BAILLIE-GIFFORD-WORLDWIDE-POSITIVE-CHANGE-FUND-B-EUR-ACC-Fonds-IE00BDCY2C68',
        'LUXEMBOURG-SELECTION-FUND-SOLAR-SUSTAINABLE-ENERGY-FUND-A2-EUR-ACC-Fonds-LU0405860593',
        'ERSTE-WWF-STOCK-ENVIRONMENT-D02-EUR-ACC-Fonds-AT0000A28E70',
        'SCHRODER-ISF-GLOBAL-ENERGY-TRANSITION-A-EUR-ACC-H-Fonds-LU2016064201',
       ]
plot_ts(X[favs+['x']])
X[favs]

In [ ]:
time_range = ['created', 'today']

X = s.get_timeseries(best_fonds['name'], time_range)
X

In [ ]:
plot_ts(X[favs+['x']])

In [ ]:
favs = ['LI0348132080', 'AT00ZUKUNFT5', 'IE00BF1T7322',
        'IE00BGGJJD81', 'IE00BK5TW834', 'IE00BZ180B88',
        'IE00BDCY2C68', 'IE00BK5TW941',
        'LU0168341575' # current portfolio
       ]

time_range = ['2020-02-19', 'today']

# X = s.get_timeseries(s.get_funds(favs), ['2020-02-19', 'today'])

fig = px.line(X, x='x', y='y', color='name')
fig.update_layout(legend={'yanchor': 'top', 'xanchor': 'left', 'y': 0, 'x': 0})
fig.show()

In [ ]:
['3-BANKEN-VERANTWORTUNG-ZUKUNFT-2024-R-EUR-ACC-Fonds-AT00ZUKUNFT5',
 'BROWN-ADVISORY-US-SUSTAINABLE-GROWTH-FUND-C-USD-ACC-Fonds-IE00BF1T7322',
'CHAMPION-ETHICAL-EQUITY-FUND-GOBAL-USD-ACC-Fonds-LI0348132080']

In [ ]:
list(s.get_funds()['name'].sort_values())

In [ ]:
# check if TER included in ts

names = ['BAILLIE-GIFFORD-GLOBAL-STEWARDSHIP-FUND-B-EUR-ACC-Fonds-IE00BGGJJD81',
 'BAILLIE-GIFFORD-GLOBAL-STEWARDSHIP-FUND-A-EUR-ACC-Fonds-IE00BK5TW834']

X = s.get_timeseries(names, ['2020-02-19', 'today'])

length = len(X)
for f in names:
    ter = s.get_funds(f, 'name')['ongoing_charges'].values[0]
    X[f] = X[f] * pd.Series(1./(1.-np.array(range(0,length))*ter/365./100.))

fig = px.line(toggle_long(X), x='x', y='y', color='name')
fig.update_layout(legend={'yanchor': 'top', 'xanchor': 'left', 'y': 0, 'x': 0})
fig.show()